1. NaNなどを適切に穴埋めする 
2. TrainとTestをconcatしたものを作る
3. 全体の中から適切にTrainとValとTestに分割してpklで保存する 

In [2]:
import gc

import pandas as pd
import numpy as np

from tools.util import Util
from tools.datahandling import DataHandling as DH

In [2]:
TRAIN_PATH = 'input/0.train.pkl'
TEST_PATH = 'input/0.test.pkl'
TEST_S_PATH = 'input/0.test_supplement.pkl'

In [3]:
df_test = pd.read_pickle(TEST_PATH)
df_test['data_set'] = int(1)
df_test['is_attributed'] = int(-999)

In [4]:
df_test_s = pd.read_pickle(TEST_S_PATH)
df_test_s["data_set"] = int(2)
df_test_s['click_id'] = int(-999)
df_test_s['is_attributed'] = int(-999)

In [5]:
df_test_s.head()

,click_id,ip,app,device,os,channel,click_time,data_set,is_attributed
0,-999,43570,3,1,18,379,2017-11-09 14:23:39,2,-999
1,-999,80528,3,1,13,379,2017-11-09 14:23:51,2,-999
2,-999,32323,3,1,13,379,2017-11-09 14:25:57,2,-999
3,-999,42887,3,1,17,379,2017-11-09 14:26:03,2,-999
4,-999,119289,58,1,30,120,2017-11-09 14:26:41,2,-999


In [6]:
df_train = pd.read_pickle(TRAIN_PATH)
df_train = df_train.drop(['attributed_time'], axis=1)
df_train["data_set"] = int(0)
df_train['click_id'] = int(-999)
df_train.head()

,ip,app,device,os,channel,click_time,is_attributed,data_set,click_id
0,83230,3,1,13,379,2017-11-06 14:32:21,0,0,-999
1,17357,3,1,19,379,2017-11-06 14:33:34,0,0,-999
2,35810,3,1,13,379,2017-11-06 14:34:12,0,0,-999
3,45745,14,1,13,478,2017-11-06 14:34:52,0,0,-999
4,161007,3,1,13,379,2017-11-06 14:35:08,0,0,-999


In [7]:
df = pd.concat([df_train, df_test, df_test_s], axis=0)
df['is_attributed'] = df['is_attributed'].astype('int32')
df['click_id'] = df['click_id'].astype('int32')

In [8]:
df.dtypes

app                       int32
channel                   int32
click_id                  int32
click_time       datetime64[ns]
data_set                  int64
device                    int32
ip                        int32
is_attributed             int32
os                        int32
dtype: object

In [9]:
df.to_pickle("input/1.all.pkl")

del df_train, df_test, df_test_s
gc.collect()

63

In [10]:
for column in df.columns:
    print(column)
    df[column].to_csv("input/1.all_" + str(column) + ".csv", header=[column], index=False)
    
del df 
gc.collect()

app
channel
click_id
click_time
data_set
device
ip
is_attributed
os


# ここからtrain, val, test分ける

In [3]:
df = pd.read_pickle("input/1.all.pkl")

In [4]:
df["day"] = df["click_time"].dt.day.astype('int')
df["hour"] = df["click_time"].dt.hour.astype('int')
df = df.drop(["click_time"], axis=1)

In [6]:
train = df[df["day"] == 8]
train = train.drop(["click_id", 'data_set'])
val   = df[(df["day"] == 9) &
           (df["hour"] >= 4)&
           (df["hour"] <= 15)]
test  = df[df["data_set"] == 1]

local = pd.concat([train, val], axis=0)

del df
gc.collect()

68

In [7]:
train.head()

,app,channel,click_id,data_set,device,ip,is_attributed,os,day,hour
68941878,23,153,-999,0,1,48240,0,13,8,0
68941879,13,477,-999,0,1,45702,0,16,8,0
68941880,12,259,-999,0,1,123214,0,13,8,0
68941881,15,265,-999,0,1,98052,0,19,8,0
68941882,6,125,-999,0,1,97903,0,18,8,0


In [8]:
val.head()

,app,channel,click_id,data_set,device,ip,is_attributed,os,day,hour
144708152,2,477,-999,0,1,38877,0,19,9,4
144708153,14,489,-999,0,1,104271,0,18,9,4
144708154,3,280,-999,0,1,92922,0,13,9,4
144708155,3,19,-999,0,1,6505,0,13,9,4
144708156,9,334,-999,0,1,19964,0,13,9,4


In [9]:
test.head()

,app,channel,click_id,data_set,device,ip,is_attributed,os,day,hour
0,9,107,0,1,1,5744,-999,3,10,4
1,9,466,1,1,1,119901,-999,3,10,4
2,21,128,2,1,1,72287,-999,19,10,4
3,15,111,3,1,1,78477,-999,13,10,4
4,12,328,4,1,1,123080,-999,13,10,4


In [11]:
train.to_pickle("input/2.initial/train.pkl")
val.to_pickle("input/2.initial/val.pkl")
test.to_pickle("input/2.initial/test.pkl")